#### PEP 657 - Enhanced error locations in tracebacks

*Unfortunately this is hard to demo in the jupyter shell*

See the [example script](./errormsg.py) instead.

In [32]:
! python errormsg.py

Traceback (most recent call last):
  File "/home/jchang/git/python311/errormsg.py", line 15, in <module>
    ticket()
    ^^^^^^^^
  File "/home/jchang/git/python311/errormsg.py", line 11, in ticket
    print(f"{city1['name']}, {city1['country']} -> {city2['name']}, {city2['country']}")
                                                                     ~~~~~^^^^^^^^^^^
KeyError: 'country'


#### PEP 678 – Enriching Exceptions with Notes

Often times it's useful to add additional information to an error as the stack unwinds, doing this usually requires explicitly creating a custom exception class that can hold this information. 

In 3.11 a new method across all Exceptions allows notes to be added to exception objects.

In [13]:
def create_city(name: str):
    raise Exception(f"Cannot create city {name = }")


def create_country(name: str):
    try:
        create_city("London")
    except Exception as e:
        e.add_note(f"Country {name = }")
        raise e


try:
    create_country("UK")
except Exception as e:
    print(e.__notes__)

["Country name = 'UK'"]


Note: jupyter stacktraces do not include notes at the time of writing, this probably will be updated in the future.

#### datetime

It is recommended to works with timezone sensitive datetimes as much as possible to avoid problems, there is now a quicker way to import the UTC timezone.

In [14]:
import time
from datetime import UTC, datetime, timedelta, timezone

# UTC is a new shortcut for timezone.utc
assert UTC is timezone.utc

start = datetime.now(UTC)
time.sleep(1)
end = datetime.now(UTC)

print((end - start) / timedelta(seconds=1))

1.000365


Be aware: it might be tempting to use `utcnow` which creates a timezone naive datetime object. This can cause bugs if we're combining with a datetime from an external source or when working with summer time adjustments. Much safer is it to let Python standard library convert the datetime implicitly.

#### WeakRef Slotted Dataclasses

In [41]:
from dataclasses import dataclass
from weakref import ref

@dataclass(slots=True)
class NormalEvent:
    id: int

n = NormalEvent(1)
r = ref(n)

TypeError: cannot create weak reference to 'NormalEvent' object

In [42]:

@dataclass(weakref_slot=True, slots=True)
class Event:
    id: int

event = Event(1)
ref(event)

<weakref at 0x7f34e7e27c40; to 'Event' at 0x7f3508624190>

#### asyncio.Barrier

In [4]:
import asyncio

barrier = asyncio.Barrier(2)


async def service(name):
    print(f"service {name} ready")
    async with barrier:
        print(f"started service {name}")


async with asyncio.TaskGroup() as g:
    g.create_task(service("a"))
    g.create_task(service("b"))

service a ready
service b ready
started service b
started service a


#### Toml

tomllib is added to Python's standard library as the toml format increases in popularity.

In [6]:
import tomllib
from pathlib import Path

pyproject_file = Path('pyproject.toml')

with pyproject_file.open('rb') as f:
    print(tomllib.load(f))

{'tool': {'poetry': {'name': 'python311', 'version': '0.1.0', 'description': '', 'authors': ['Your Name <you@example.com>'], 'dependencies': {'python': '^3.11', 'ipykernel': '^6.13.0', 'nbconvert': '^6.5.0', 'typing-extensions': '^4.2.0', 'black': '^22.3.0'}, 'dev-dependencies': {'ipykernel': '^6.13.0'}}}, 'build-system': {'requires': ['poetry-core>=1.0.0'], 'build-backend': 'poetry.core.masonry.api'}}
